# Generate ELK Snapshot on Meas Node
### This notebook creates an ELK snapshot on the measurement node and helps you copy it to a backup destinaton

### Prerequisites
- Slice needs to be created: [Prepare a Slice](../prepare_a_slice.ipynb)
- Slice needs to be instrumentized: [Instrumentize a Slice](../instrumentize_a_slice.ipynb)
---

## Setup
#### Imports

In [ ]:
import os
import json
import traceback
import datetime
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

In [ ]:
import importlib
from mflib.data_transfer import ElkExporter

#### Set Slice Info

In [ ]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

#### Create MFLib and ELK export service objects

In [ ]:
mf = MFLib(slice_name)
meas_node_name = mf.measurement_node_name

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")
    
try:
    elk_tools = ElkExporter(slice_name=slice_name, local_storage_directory="/tmp/mflib", node_name="meas-node")
except Exception as e:
    print(f"Fail: {e}")

#### Display the current Measurement Node

In [ ]:
print(meas_node)

---
## Create an ELK Snapshot

#### Choose name for the repository and snapshots you will create

In [ ]:
repository_name= "backup_repository"
snapshot_name =  "snapshot_test"

#### Create a snapshot repository

In [ ]:
elk_tools.create_repository(repository_name=repository_name)

#### View indices

In [ ]:
elk_tools.view_indices()

#### Create snapshot

In [ ]:
elk_tools.create_snapshot(repository_name=repository_name, snapshot_name=snapshot_name)

#### Check that the snapshot is in the created repository 

In [ ]:
elk_tools.view_repository(repository_name=repository_name)

#### View the snapshot in the repository

In [ ]:
elk_tools.view_snapshot(repository_name="backup_repository", snapshot_name=snapshot_name)

---
## Export snapshot to Measurement node as tar file

In [ ]:
elk_tools.export_snapshot_tar(snapshot_name=snapshot_name)

---
## Copy snapshot off Measurement node

### Option 1: Copy to a local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# The ssh config file location on your local device
ssh_config = "~/.ssh/fabric/ssh_config"

# The fabric slice private key file on your local device
private_key = "~/.ssh/fabric/slice_key"

# The Measurement Node username
username = mf.slice_username

# The Measurement Node IP Address
ip = mf.meas_node_ip

# Destination to store snapshot file on your local device
local_file_destination = "/your/path/here"

# Snapshot file name (Keep commented out to use the last used in this notebook)
#snapshot_name = "snapshot"

# Create and print command
scp_command = f"scp -F {ssh_config} -i {private_key} {username}@\[{ip}]:/home/mfuser/services/elk/files/snapshots/{snapshot_name}.tar {local_file_destination}"
print(scp_command)

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

### Option 2: Copy to Jupyter Hub server

##### **CAUTION** The JH servers have very limited storage. If you are doing this method, make sure you do not download large or multiple snapshot files.
##### Fill in the variables below then run to download file to JH server

In [ ]:
# Snapshot file name (Keep commented out to use the last used in this notebook)
#snapshot_name = "snapshot"

# Destination to store snapshot file on your local device
local_file_destination = f"./snapshots/{snapshot_name}.tar"


meas_node.download_file(local_file_destination, f"/home/mfuser/services/elk/files/snapshots/{snapshot_name}.tar")

---

# Export complete

For import instructions, refer to [Elk Import](./elk_import.ipynb)